In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('ggplot')     # Стиль графиков, можно закомментировать
plt.rcParams['figure.figsize'] = (10, 7.5)              

import sklearn
import sklearn.datasets as datasets
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [ ]:
data_home = datasets.fetch_openml(name='credit-g')

/usr/local/lib/python3.8/dist-packages/sklearn/datasets/_openml.py:417: UserWarning: Multiple active versions of the dataset matching the name credit-g exist. Versions may be fundamentally different, returning version 1.
  warn(


In [ ]:
Predictors_home = data_home.get('data')
Target_home = data_home.get('target')
Predictors_home = Predictors_home.drop(['own_telephone'], axis=1)

In [ ]:
Predictors_home['new'] = Target_home

In [ ]:
Predictors_home['new'].value_counts()

good    700
bad     300
Name: new, dtype: int64

In [ ]:
pr_good = Predictors_home[Predictors_home['new'] == 'good'][:450]
pr_bad = Predictors_home[Predictors_home['new'] == 'bad'][:250]
Predictors = pr_good.append(pr_bad, ignore_index=True)
from sklearn.utils import shuffle
Predictors = shuffle(Predictors)
answers = Predictors['new']
Predictors = Predictors.drop('new', axis=1)
Predictors_home = Predictors_home.drop('new', axis=1)

In [ ]:
Predictors = pd.get_dummies(Predictors)
Predictors_home = pd.get_dummies(Predictors_home)

In [ ]:
X_train, X_test, Y_train, Y_test = Predictors, Predictors_home[701:], answers, Target_home[701:]

In [ ]:
X_train.columns

Index(['duration', 'credit_amount', 'installment_commitment',
       'residence_since', 'age', 'existing_credits', 'num_dependents',
       'checking_status_<0', 'checking_status_0<=X<200',
       'checking_status_>=200', 'checking_status_no checking',
       'credit_history_no credits/all paid', 'credit_history_all paid',
       'credit_history_existing paid', 'credit_history_delayed previously',
       'credit_history_critical/other existing credit', 'purpose_new car',
       'purpose_used car', 'purpose_furniture/equipment', 'purpose_radio/tv',
       'purpose_domestic appliance', 'purpose_repairs', 'purpose_education',
       'purpose_vacation', 'purpose_retraining', 'purpose_business',
       'purpose_other', 'savings_status_<100', 'savings_status_100<=X<500',
       'savings_status_500<=X<1000', 'savings_status_>=1000',
       'savings_status_no known savings', 'employment_unemployed',
       'employment_<1', 'employment_1<=X<4', 'employment_4<=X<7',
       'employment_>=7', 'per

In [ ]:
num_features = X_train.select_dtypes("float64").columns

In [ ]:
X_train.shape, X_test.shape

((700, 61), (299, 61))

In [ ]:
scaler = StandardScaler()
X_train[num_features] = scaler.fit_transform(X_train[num_features])
X_test[num_features] = scaler.transform(X_test[num_features])
X_train = X_train.to_numpy()
Y_train = Y_train.to_numpy()
X_test = X_test.to_numpy()
Y_test = Y_test.to_numpy()

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [ ]:
clf = LogisticRegression() 
clf.fit(X_train, Y_train)
clf.score(X_test, Y_test)

0.782608695652174

In [ ]:
y_hat = clf.predict(X_test)
print(classification_report(Y_test, y_hat))

              precision    recall  f1-score   support

         bad       0.64      0.67      0.66        92
        good       0.85      0.83      0.84       207

    accuracy                           0.78       299
   macro avg       0.75      0.75      0.75       299
weighted avg       0.79      0.78      0.78       299



In [ ]:
number_of_neighbors = np.arange(3, 25, 2)
model_KNN = KNeighborsClassifier()
params = {"n_neighbors": number_of_neighbors}

grid_search = GridSearchCV(estimator = model_KNN, 
                           param_grid = params, 
                           cv = 6, 
                           scoring = "f1_macro")

In [ ]:
grid_search.fit(X_train, Y_train)

GridSearchCV(cv=6, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23])},
             scoring='f1_macro')

In [ ]:
grid_search.best_score_

0.6215986699665028

In [ ]:
preds_knn=grid_search.predict(X_test)

In [ ]:
f1_score(Y_test, preds_knn, average = "macro")

0.6757753198872263

In [ ]:
print(classification_report(Y_test, preds_knn))

              precision    recall  f1-score   support

         bad       0.57      0.51      0.54        92
        good       0.79      0.83      0.81       207

    accuracy                           0.73       299
   macro avg       0.68      0.67      0.68       299
weighted avg       0.73      0.73      0.73       299

